# Field Decomp


In [ ]:
%%bigquery
select * from retails_raw.parts limit 5

Query is running:   0%|          |

Downloading:   0%|          |

,p_partkey,p_type,p_size,p_brand,p_name,p_container,p_mfgr,p_retailprice,p_comment,load_time
0,316,LARGE PLATED NICKEL,1,Brand#11,orchid mint blush khaki metallic,WRAP CAN,Manufacturer#1,1216.31,ironic packages wa,2024-02-06 18:09:51.654365+00:00
1,1577,SMALL POLISHED COPPER,1,Brand#11,violet floral sienna ghost green,MED JAR,Manufacturer#1,1478.57,carefully speci,2024-02-06 18:09:51.654365+00:00
2,1676,MEDIUM POLISHED TIN,1,Brand#11,lawn black blanched slate tomato,WRAP PACK,Manufacturer#1,1577.67,slyly reg,2024-02-06 18:09:51.654365+00:00
3,2633,PROMO PLATED NICKEL,1,Brand#11,drab lemon plum rose salmon,JUMBO BAG,Manufacturer#1,1535.63,slyly exp,2024-02-06 18:09:51.654365+00:00
4,6358,ECONOMY POLISHED TIN,1,Brand#11,purple plum blush sky lavender,JUMBO BAG,Manufacturer#1,1264.35,blithely even exc,2024-02-06 18:09:51.654365+00:00


##Parts

In [ ]:
%%bigquery
create or replace table retails_stg.Parts as
  select cast(p_partkey as INT64)  as p_partkey,
  concat(p_type_array[OFFSET(0)], ' ', p_type_array[OFFSET(1)]) as size_description,
  p_type_array[OFFSET(2)] as material, cast(p_size as INT64) as p_size, p_brand, p_name,  p_container, p_mfgr, cast(p_retailprice as FLOAT64) as p_retailprice, p_comment,  'bird' as data_source, load_time
  from
    (select p_partkey, split(p_type, ' ') as p_type_array, p_size, p_brand, p_name,  p_container, p_mfgr, p_retailprice, p_comment,  'bird' as data_source, load_time
      from retails_raw.parts)

Query is running:   0%|          |

""


# Parts Verification

In [ ]:
%%bigquery
select (select count(*) from retails_raw.parts) as raw_count,
  (select count(*) from retails_stg.Parts) as intermediate_stg_count

Query is running:   0%|          |

Downloading:   0%|          |

,raw_count,intermediate_stg_count
0,200000,200000


##Supplier


In [ ]:
%%bigquery
create or replace table retails_stg.Supplier as
  select
  cast(s_suppkey as INT64)  as s_suppkey,
  s_nationkey,
  s_comment,
  s_name,
  s_address,
  s_phone_array[OFFSET(0)] as area_code,
  concat(s_phone_array[OFFSET(1)], '-', s_phone_array[OFFSET(2)]) as phone_number,
  cast(s_acctbal AS FLOAT64) AS s_acctbal,
  'bird' AS data_source,
  Date as load_time
  from
    (select s_suppkey, s_nationkey, s_comment, s_name,  s_address, split(s_phone, '-') as s_phone_array, s_acctbal,  'bird' as data_source, Date
      from retails_raw.supplier)

Query is running:   0%|          |

""


# Supplier Verification

In [ ]:
%%bigquery
select (select count(*) from retails_raw.supplier) as raw_count,
  (select count(*) from retails_stg.Supplier) as intermediate_stg_count

Query is running:   0%|          |

Downloading:   0%|          |

,raw_count,intermediate_stg_count
0,10000,10000


##Sales

In [ ]:
%%bigquery
create or replace table retails_stg.Sales as
  select
  cast(Store as INT64)  as Store_ID,
  cast(Dept as INT64)  as Dept_ID,
  Date,
  date_array[OFFSET(0)] as Day,
  date_array[OFFSET(1)] as Month,
  date_array[OFFSET(2)] as Year,
  cast(Weekly_Sales as FLOAT64) as Weekly_Sales,
  cast(IsHoliday as BOOLEAN) as IsHoliday,
  'kaggle' AS data_source,
  load_time
  from
    (select Store, Dept, Date, split(Date, '/') as date_array, Weekly_Sales, IsHoliday, 'kaggle' as data_source, load_time
[ ]

      from retails_raw.sales)

Query is running:   0%|          |

""


# Sales Verification

In [ ]:
%%bigquery
select (select count(*) from retails_raw.sales) as raw_count,
  (select count(*) from retails_stg.Sales) as intermediate_stg_count

Query is running:   0%|          |

Downloading:   0%|          |

,raw_count,intermediate_stg_count
0,421570,421570


##Transactions

In [ ]:
%%bigquery
create or replace table retails_stg.Transactions as
  select
  cast(id as INT64)  as Transaction_ID,
  date,
  date_array[OFFSET(2)] as Day,
  date_array[OFFSET(1)] as Month,
  date_array[OFFSET(0)] as Year,
  cast(store_nbr as INT64)  as store_nbr,
  family,
  cast(sales as FLOAT64) as sales_amount,
  case when onpromotion = '0' then false else true end as on_promotion,
  'kaggle' AS data_source,
  load_time
  from
    (select id, date, split(date, '-') as date_array, store_nbr, family, sales, onpromotion, 'kaggle' as data_source, load_time
      from retails_raw.transactions)

Query is running:   0%|          |

""


# Transaction Verifcation

##Partsupp

In [ ]:
%%bigquery
create or replace table retails_stg.Partsupp as
  select
  cast(ps_partkey as INT64)  as ps_partkey,
  cast(ps_suppkey as INT64)  as ps_suppkey,
  cast(ps_supplycost as FLOAT64) as ps_supplycost,
  cast(ps_availqty as INT64) as ps_availqty,
  ps_comment,
  'bird' as data_source,
  load_time
  from
    (select ps_partkey, ps_suppkey, ps_supplycost, ps_availqty, ps_comment, 'bird' as data_source, load_time
      from retails_raw.partsupp)

Query is running:   0%|          |

""


# Partsupp Verification

In [ ]:
%%bigquery
select (select count(*) from retails_raw.transactions) as raw_count,
  (select count(*) from retails_stg.Transactions) as intermediate_stg_count

Query is running:   0%|          |

Downloading:   0%|          |

,raw_count,intermediate_stg_count
0,3000888,3000888


In [ ]:
%%bigquery
select (select count(*) from retails_raw.partsupp) as raw_count,
  (select count(*) from retails_stg.Partsupp) as intermediate_stg_count

Query is running:   0%|          |

Downloading:   0%|          |

,raw_count,intermediate_stg_count
0,800000,800000


##Minerals

In [ ]:
%%bigquery
create or replace table retails_stg.Mineral as
  select
  site_name,
  cast(latitude as FLOAT64)  as latitude,
  cast(longitude as FLOAT64)  as longitude,
  region,
  country,
  state,
  county,
  com_type,
  commod1,
  commod2,
  commod3,
  'kaggle' as data_source,
  load_time
  from
    (select site_name, latitude, longitude, region, country, state, county, com_type, commod1, commod2, commod3, 'kaggle' as data_source, load_time
      from retails_raw.mineral)

Query is running:   0%|          |

""


# Minerals Verification

In [ ]:
%%bigquery
select (select count(*) from retails_raw.mineral) as raw_count,
  (select count(*) from retails_stg.Mineral) as intermediate_stg_count

Query is running:   0%|          |

Downloading:   0%|          |

,raw_count,intermediate_stg_count
0,304621,304621


##Mrds

In [ ]:
%%bigquery
create or replace table retails_stg.Mrds as
  select
  cast(dep_id as INT64)  as Dep_ID,
  url,
  mrds_id,
  mas_id,
  site_name,
  cast(latitude as FLOAT64)  as latitude,
  cast(longitude as FLOAT64)  as longitude,
  region,
  country,
  state,
  county,
  com_type,
  commod1,
  commod2,
  commod3,
  'usgs' as data_source,
  load_time
  from
    (select dep_id, url, mrds_id, mas_id, site_name, latitude, longitude, region, country, state, county, com_type, commod1, commod2, commod3, 'usgs' as data_source, load_time
      from retails_raw.mrds)

Query is running:   0%|          |

""


In [ ]:
%%bigquery
select (select count(*) from retails_raw.mrds) as raw_count,
  (select count(*) from retails_stg.Mrds) as intermediate_stg_count

Query is running:   0%|          |

Downloading:   0%|          |

,raw_count,intermediate_stg_count
0,304632,304632


#Primary Keys

##Parts PK

In [ ]:
%%bigquery
alter table retails_stg.Parts
  add primary key (p_partkey) not enforced;

Query is running:   0%|          |

""


In [ ]:
%%bigquery
select p_partkey, count(*) duplicate_records
from retails_stg.Parts
group by p_partkey
having count(*) > 1
order by count(*) desc

Query is running:   0%|          |

Downloading: |          |

,p_partkey,duplicate_records


##Supplier PK

In [ ]:
%%bigquery
alter table retails_stg.Supplier
  add primary key (s_suppkey) not enforced;

Query is running:   0%|          |

""


In [ ]:
%%bigquery
select s_suppkey, count(*) duplicate_records
from retails_stg.Supplier
group by s_suppkey
having count(*) > 1
order by count(*) desc

Query is running:   0%|          |

Downloading: |          |

,s_suppkey,duplicate_records


##Sales PK

In [ ]:
%%bigquery
alter table retails_stg.Sales
  add primary key (Store_ID, Dept_ID, Date) not enforced;

Query is running:   0%|          |

""


In [ ]:
%%bigquery
select Store_ID, Dept_ID, Date, count(*) duplicate_records
from retails_stg.Sales
group by Store_ID, Dept_ID, Date
having count(*) > 1
order by count(*) desc

Query is running:   0%|          |

Downloading: |          |

,Store_ID,Dept_ID,Date,duplicate_records


##Transactions PK

In [ ]:
%%bigquery
alter table retails_stg.Transactions
  add primary key (Transaction_ID) not enforced;

Query is running:   0%|          |

""


In [ ]:
%%bigquery
select Transaction_ID, count(*) duplicate_records
from retails_stg.Transactions
group by Transaction_ID
having count(*) > 1
order by count(*) desc

Query is running:   0%|          |

Downloading: |          |

,Transaction_ID,duplicate_records


##Partsupp PK

In [ ]:
%%bigquery
alter table retails_stg.Partsupp
  add primary key (ps_partkey, ps_suppkey) not enforced;

Query is running:   0%|          |

""


In [ ]:
%%bigquery
select ps_partkey, ps_suppkey, count(*) duplicate_records
from retails_stg.Partsupp
group by ps_partkey, ps_suppkey
having count(*) > 1
order by count(*) desc

Query is running:   0%|          |

Downloading: |          |

,ps_partkey,ps_suppkey,duplicate_records


##Mineral PK

In [ ]:
%%bigquery
alter table retails_stg.Mineral
  add primary key (site_name) not enforced;

Query is running:   0%|          |

""


In [ ]:
%%bigquery
select site_name, count(*) duplicate_records
from retails_stg.Mineral
group by site_name
having count(*) > 1
order by count(*) desc

Query is running:   0%|          |

Downloading:   0%|          |

,site_name,duplicate_records
0,Unknown,8617
1,Gravel Pit,4191
2,Unnamed Prospect,4016
3,Unk,3150
4,Unnamed Pit,2313
...,...,...
33039,Levack East Mine,2
33040,"East Mine, Falconbridge Limited",2
33041,Onaping-Craig Mine,2
33042,Fecunis-North Mine,2


##Mrds PK

In [ ]:
%%bigquery
alter table retails_stg.Mrds
  add primary key (Dep_ID) not enforced;

Query is running:   0%|          |

""


In [ ]:
%%bigquery
select Dep_ID, count(*) duplicate_records
from retails_stg.Mrds
group by Dep_ID
having count(*) > 1
order by count(*) desc

Query is running:   0%|          |

Downloading: |          |

,Dep_ID,duplicate_records
